In [1]:
import numpy as np

In [ ]:
def create_valid_sudoku(block_rows, block_cols):
    
    
class Sudoku(np.ndarray):
    
    def __init__(self):
        self.(shape=(3,3,3,3), dtype=np.int16)
    
    def self.get_block(block=None)

In [2]:
type(np.ndarray(shape=(3,3,3,3), dtype=np.int16))

numpy.ndarray

In [3]:
import numpy as np

class Sudoku(np.ndarray):
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        if obj.shape != (3, 3, 3, 3):
            raise ValueError("Input array must have shape (3, 3, 3, 3)")
        return obj

    def __array_finalize__(self, obj):
        if obj is None:
            return

    def __getitem__(self, item):
        if isinstance(item, tuple):
            if len(item) == 4:
                block_row, block_col, row, col = item
                return super().__getitem__((block_row, block_col, row, col))
            elif len(item) == 2:
                row, col = item
                row_block = row // 3
                col_block = col // 3
                inner_row = row % 3
                inner_col = col % 3
                return super().__getitem__((row_block, col_block, inner_row, inner_col)).flatten()
        return super().__getitem__(item)

    def __setitem__(self, item, value):
        if isinstance(item, tuple):
            if len(item) == 4:
                block_row, block_col, row, col = item
                super().__setitem__((block_row, block_col, row, col), value)
            elif len(item) == 2:
                row, col = item
                row_block = row // 3
                col_block = col // 3
                inner_row = row % 3
                inner_col = col % 3
                super().__setitem__((row_block, col_block, inner_row, inner_col), value)
        else:
            super().__setitem__(item, value)


# Example usage:
sudoku_grid = np.zeros((3, 3, 3, 3), dtype=int)
sudoku = Sudoku(sudoku_grid)

# Set a value using blockRow, blockCol, row, col coordinates
sudoku[1, 2, 0, 1] = 5

# Set a value using simple row and column indexing
sudoku[5, 7] = 2

print("Sudoku grid:")
print(sudoku)

# Get values using blockRow, blockCol, row, col coordinates
print("\nValue at (1, 2, 0, 1):", sudoku[1, 2, 0, 1])

# Get values using simple row and column indexing
print("Value at (5, 7):", sudoku[5, 7])


Sudoku grid:
[[[[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]]]


 [[[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 5 0]
   [0 0 0]
   [0 2 0]]]


 [[[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]]]]

Value at (1, 2, 0, 1): 5
Value at (5, 7): [2]


In [173]:
import numpy as np

class Sudoku(np.ndarray):
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        if obj.shape != (3, 3, 3, 3):
            raise ValueError("Input array must have shape (3, 3, 3, 3)")
        return obj

    def __array_finalize__(self, obj):
        if obj is None:
            return

    def __getitem__(self, item):
        if isinstance(item, tuple) and len(item) == 4: return super(Sudoku, self).__getitem__(item)
        raise IndexError("Wrong indexing for Sudoku: must need 4 indices")

    def get(self, row, col):
        row_block = row // 3
        inner_row = row % 3
        col_block = col // 3
        inner_col = col % 3
        return super(Sudoku, self).__getitem__((row_block, col_block, inner_row, inner_col))
    
    def get_row(self, row):
        row_block = row // 3
        inner_row = row % 3
    
    def get_col(self,col):
        pass
        

    def __setitem__(self, item, value):
        if isinstance(item, tuple) and len(item) == 4: super().__setitem__(item, value)
        raise IndexError("Wrong indexing for Sudoku: must need 4 indices")

    def __repr__(self):
        try:
            lines = []
            for block_row in range(3):
                for row in range(3):
                    line = ""
                    for block_col in range(3):
                        for col in range(3):
                            value = self[block_row, block_col, row, col]
                            line += f" {value} " if value != 0 else " 0 "
                        if block_col < 2:
                            line += "|"
                    lines.append(line)
                if block_row < 2:
                    lines.append("---------+---------+---------")
            return "\n".join(lines)
        except:
            return super().__repr__()
    
    def __str__(self):
        return self.__repr__()


# Example usage:
sudoku_grid = np.array([
    [[[5, 3, 0], [6, 0, 0], [0, 9, 8]],
    [[0, 7, 0], [1, 9, 5], [0, 0, 0]],
    [[0, 0, 0], [0, 0, 0], [0, 6, 0]]],
    [[[8, 0, 0], [4, 0, 0], [7, 0, 0]],
    [[0, 6, 0], [8, 0, 3], [0, 2, 0]],
    [[0, 0, 3], [0, 0, 1], [0, 0, 6]]],
    [[[0, 6, 0], [0, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [4, 1, 9], [0, 8, 0]],
    [[2, 8, 0], [0, 0, 5], [6, 0, 0]]]
])

sudoku = Sudoku(sudoku_grid)
print("Sudoku grid:")
print(sudoku)


Sudoku grid:
 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 


In [63]:
sudoku

 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 

In [67]:
print(sudoku.get(0, 0))

3


In [65]:
sudoku_grid[0, 0, :, :] = 3

In [66]:
sudoku

 3  3  3 | 0  7  0 | 0  0  0 
 3  3  3 | 1  9  5 | 0  0  0 
 3  3  3 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 

In [73]:
sudoku.reshape(9, 9)

IndexError: Wrong indexing for Sudoku: must need 4 indices

In [40]:
print(sudoku[slice(None, None, None), slice(None, None, None), 0, 0])

<class '__main__.Sudoku'> (slice(None, None, None), slice(None, None, None), 0, 0) 4
slice(None, None, None) slice(None, None, None) 0 0
<class '__main__.Sudoku'> (-3, -3) 2


ValueError: Bad Input

In [39]:
sudoku

<class '__main__.Sudoku'> (0, 0, 0, 0) 4
0 0 0 0
<class '__main__.Sudoku'> (0, 0, 0, 1) 4
0 0 0 1
<class '__main__.Sudoku'> (0, 0, 0, 2) 4
0 0 0 2
<class '__main__.Sudoku'> (0, 1, 0, 0) 4
0 1 0 0
<class '__main__.Sudoku'> (0, 1, 0, 1) 4
0 1 0 1
<class '__main__.Sudoku'> (0, 1, 0, 2) 4
0 1 0 2
<class '__main__.Sudoku'> (0, 2, 0, 0) 4
0 2 0 0
<class '__main__.Sudoku'> (0, 2, 0, 1) 4
0 2 0 1
<class '__main__.Sudoku'> (0, 2, 0, 2) 4
0 2 0 2
<class '__main__.Sudoku'> (0, 0, 1, 0) 4
0 0 1 0
<class '__main__.Sudoku'> (0, 0, 1, 1) 4
0 0 1 1
<class '__main__.Sudoku'> (0, 0, 1, 2) 4
0 0 1 2
<class '__main__.Sudoku'> (0, 1, 1, 0) 4
0 1 1 0
<class '__main__.Sudoku'> (0, 1, 1, 1) 4
0 1 1 1
<class '__main__.Sudoku'> (0, 1, 1, 2) 4
0 1 1 2
<class '__main__.Sudoku'> (0, 2, 1, 0) 4
0 2 1 0
<class '__main__.Sudoku'> (0, 2, 1, 1) 4
0 2 1 1
<class '__main__.Sudoku'> (0, 2, 1, 2) 4
0 2 1 2
<class '__main__.Sudoku'> (0, 0, 2, 0) 4
0 0 2 0
<class '__main__.Sudoku'> (0, 0, 2, 1) 4
0 0 2 1
<class '__main__.Sud

 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 

In [216]:
print("\n\n".join("\n".join("x".join(str(k) for k in j) for j in i) for i in sudoku_grid.reshape(3,9,3).T.reshape(3,3,3,3)))

[ 0 27 54]x[ 3 30 57]x[ 6 33 60]
[ 9 36 63]x[12 39 66]x[15 42 69]
[18 45 72]x[21 48 75]x[24 51 78]

[ 1 28 55]x[ 4 31 58]x[ 7 34 61]
[10 37 64]x[13 40 67]x[16 43 70]
[19 46 73]x[22 49 76]x[25 52 79]

[ 2 29 56]x[ 5 32 59]x[ 8 35 62]
[11 38 65]x[14 41 68]x[17 44 71]
[20 47 74]x[23 50 77]x[26 53 80]


In [197]:
sudoku_grid = np.array(range(81)).reshape(3,3,3,3) 

In [198]:
print("\n\n".join("\n".join("x".join(str(k) for k in j) for j in i) for i in sudoku_grid.reshape(3,3,3,3)))

[0 1 2]x[3 4 5]x[6 7 8]
[ 9 10 11]x[12 13 14]x[15 16 17]
[18 19 20]x[21 22 23]x[24 25 26]

[27 28 29]x[30 31 32]x[33 34 35]
[36 37 38]x[39 40 41]x[42 43 44]
[45 46 47]x[48 49 50]x[51 52 53]

[54 55 56]x[57 58 59]x[60 61 62]
[63 64 65]x[66 67 68]x[69 70 71]
[72 73 74]x[75 76 77]x[78 79 80]


In [218]:
Sudoku(sudoku_grid)

 0  1  2 | 9  10  11 | 18  19  20 
 3  4  5 | 12  13  14 | 21  22  23 
 6  7  8 | 15  16  17 | 24  25  26 
---------+---------+---------
 27  28  29 | 36  37  38 | 45  46  47 
 30  31  32 | 39  40  41 | 48  49  50 
 33  34  35 | 42  43  44 | 51  52  53 
---------+---------+---------
 54  55  56 | 63  64  65 | 72  73  74 
 57  58  59 | 66  67  68 | 75  76  77 
 60  61  62 | 69  70  71 | 78  79  80 